In [155]:
import warnings
warnings.simplefilter('ignore')

In [156]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [157]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [158]:
import pandas as pd
import numpy as np
# %matplotlib inline
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as mae

In [159]:
# load libraries and set plot parameters

# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('pdf', 'png')
# plt.rcParams['savefig.dpi'] = 75

# plt.rcParams['figure.autolayout'] = False
# plt.rcParams['figure.figsize'] = 10, 6
# plt.rcParams['axes.labelsize'] = 18
# plt.rcParams['axes.titlesize'] = 20
# plt.rcParams['font.size'] = 16
# plt.rcParams['lines.linewidth'] = 2.0
# plt.rcParams['lines.markersize'] = 8
# plt.rcParams['legend.fontsize'] = 14

# plt.rcParams['text.usetex'] = True
# plt.rcParams['font.family'] = "serif"
# plt.rcParams['font.serif'] = "cm"
# plt.rcParams['text.latex.preamble'] = "\usepackage{subdepth}, \usepackage{type1cm}"

In [160]:
import roboclimate.util as rutil
import datetime as dt
import data_explorer as rdq
import config as rconf

# Table of Contents

* [Introduction](#intro)
* [Models](#models)
* [Metrics](#metrics)
* [Data collection](#data_collection)
* [Data analysis](#data_analysis)
    * [Data quality](#data_quality)
    * [Actual vs Forecast](#actual_vs_forecast)
    * [Metrics comparison](#metrics_comparison)
    * [Cities comparison](#cities_comparison)

<a id="intro"></a>
# Introduction

Have you ever complained about the weatherman failing to predict the weather correctly?

That's the question we aim to answer here: how reliable are the weather forecasts?

To do so we compare the accuracy of different weather models ranging from a naive approach to sophisticated meteorological models.

The scope of this analysis is restricted to the **temperature** in five locations around the world, namely:

- London
- Madrid
- Sydney
- New York
- Sao Paulo

<a id="models"></a>
# Models


### Naive forecast

Consists in assuming that the next value is the same as the one of the last period.

The tricky part is to identify what the last value is. For instance, if we measure the temperature
every 3 hours and we want to predict the temperature today at 3pm, what is the last value: today's temperature at 12pm, yesterday's temperature at 3pm or maybe last year's temperature on the same day at 3pm?


### Average values

Following with the previous example, we could use the average of the temperature over the last 20
years on the same day at 3pm


### Meteorological models

Provided by [OpenWeather](https://openweathermap.org/technology)

<a id="metrics"></a>
# Metrics

Metrics are used to evaluate the accuracy of the models' predictions when compared to the actual values.


## Mean absolute scaled error (MASE)

Mean absolute scaled error is a measure of the precision of a model compared to the naive forecast.

It is calculated as the mean absolute error of the forecast values predicted by the model divided by the mean absolute error of the naive forecast.

Values greater than one indicate that the naive method performs better than the more sophisticated forecast methods.

### Example

If $t^1$ is the 1-day forecast made on _2020-11-22 18:00_ for _2020-11-23 18:00_, the naive forecast is the actual temperature on _2020-11-22 18:00_.

Equally, if $t^2$ is the 2-day forecast made on _2020-11-21 18:00_ for _2020-11-23 18:00_, the naive forecast is the actual temperature on _2020-11-21 18:00_.

And so on for $t^3, t^4$ and $t^5$.

Considering that everyday contains 8 forecasts (one every 3 hours), the mean absolute scaled error $mase^j$ corresponding to the _j-day_ forecast can be expressed as (where $t$ is the actual temperature):

$$
mase^j=\frac{\displaystyle\sum_{i} \lvert {t_i - t_i^j} \rvert}{\displaystyle\sum_{i} \lvert {t_i - t_{i-8j}} \rvert} 
$$

In other words, the _mase_ estimates how good the forecast $t^j$ is compared to just assuming that the temperature in $j$ days time will be the same as when the forecast $t^j$ is made.


### 1-year mase

For those locations with enough data, we also calculate the _mase_ based on the temperature 1 year ago.



## Mean absolute error (MAE)

Average of the absolute value of the errors (the errors being the differences between predicted and real values)

## Root mean squared error (RMSE)

Square root of the average of the square of the errors

It weighs outliers more heavily than MAE as a result of the squaring of each term, which effectively weighs large errors more heavily than small ones

## Median absolute error (MEDAE)

Median of the absolute value of the errors.

It is robust to outliers

<a id="data_collection"></a>
# Data collection


This page collects data about the actual weather and the meteorological forecast for the following 5 days.

This data is obtained from [OpenWeather](https://openweathermap.org) through the endpoints:

- current weather data
- 5 day forecast

Given that the 5 day forecast endpoint only provides data every 3 hours (00:00, 03:00, 06:00, 09:00, 12:00, 15:00, 18:00, 21:00), those are also the data points for which the current weather data is read.

As a result, for every given temperature there are 5 forecasts: $t_5, t_4, t_3, t_2$ and $t_1$, where $t_n$ is the forecast made $n$ days before.

<a id="data_analysis"></a>
# Data analysis

In [161]:
city_name = {"london":"London", "madrid":"Madrid", "saopaulo":"Sao Paulo", "sydney":"Sydney", "newyork":"New York", "asuncion": "Asuncion", "tokyo": "Tokyo", "moscow": "Moscow", "lagos":"Lagos","nairobi":"Nairobi"}

<a id="data_quality"></a>
## Data quality

First step of the data analysis is to check the data quality

### Missing temperatures
Finds dts for which the temperature was not recorded. As per today (2021-01-21), only New York failed to record a temperature, the one corresponding to `1593475200 = 2020-06-30T00:00:00`

In [162]:
for city in rconf.cities.values():
    print(city.name)
    print(rdq.missing_weather_datapoints(city, end_dt=dt.datetime.combine(dt.date.today(), dt.time.min, dt.timezone.utc)))

london
      temp  pressure  humidity  wind_speed  wind_deg          dt today_x  \
4572   NaN       NaN       NaN         NaN       NaN  1624287600     NaN   
6238   NaN       NaN       NaN         NaN       NaN  1642280400     NaN   
6239   NaN       NaN       NaN         NaN       NaN  1642291200     NaN   
6240   NaN       NaN       NaN         NaN       NaN  1642302000     NaN   
6241   NaN       NaN       NaN         NaN       NaN  1642312800     NaN   
...    ...       ...       ...         ...       ...         ...     ...   
9683   NaN       NaN       NaN         NaN       NaN  1679475600     NaN   
9684   NaN       NaN       NaN         NaN       NaN  1679486400     NaN   
9685   NaN       NaN       NaN         NaN       NaN  1679497200     NaN   
9686   NaN       NaN       NaN         NaN       NaN  1679508000     NaN   
9687   NaN       NaN       NaN         NaN       NaN  1679518800     NaN   

         today_y      _merge  
4572  2021-06-21  right_only  
6238  2022-01-15  

### Unexpected temperatures

Finds temperatures recorded at the wrong dts, namely, any dt other than the ones corresponding to the times 0,3,6,9,12,15,18,21 of each day.

Again, the only case is New York, and now it is clear why we found a missing temperature previously. The temperature, instead was not recorded at `1593475200` but 2 seconds later at `1593475202`

In [163]:
for city in rconf.cities.values():
    print(city.name)
    print(rdq.unexpected_weather_datapoints(city, end_dt=dt.datetime.combine(dt.date.today() + dt.timedelta(1), dt.time.min, dt.timezone.utc)))

london
Empty DataFrame
Columns: [temp, pressure, humidity, wind_speed, wind_deg, dt, today_x, today_y, _merge]
Index: []
madrid
Empty DataFrame
Columns: [temp, pressure, humidity, wind_speed, wind_deg, dt, today_x, today_y, _merge]
Index: []
saopaulo
       temp  pressure  humidity  wind_speed  wind_deg          dt     today_x  \
4666  21.22       997        90         3.6       100  1678394359  2023-03-09   

     today_y     _merge  
4666     NaN  left_only  
sydney
       temp  pressure  humidity  wind_speed  wind_deg          dt     today_x  \
1951  19.60      1015        56        2.06        30  1612969258  2021-02-10   
1964  33.76      1004        27        7.72       330  1613109633  2021-02-12   
1998  20.26      1022        94        5.66       120  1613476843  2021-02-16   

     today_y     _merge  
1951     NaN  left_only  
1964     NaN  left_only  
1998     NaN  left_only  
newyork
Empty DataFrame
Columns: [temp, pressure, humidity, wind_speed, wind_deg, dt, today_x, tod

### Temperatures without the 5 forecasts
Finds dts for which all five forecasts were not made.
Obviously, the first 5 days after starting to take measurements cannot have all the 5 previous forecasts.

In addition, the following cities failed to collect all 5 forecasts during the periods:

- Madrid, between 2020-10-06 and 2020-10-10
- Sao Paulo, betwen 2020-10-26 and 2020-10-30

That means that Madrid must have failed to record the forecast on 2020-10-05 and Sao Paulo on 2020-10-25 

In [164]:
for city in rconf.cities.values():
    print(city.name)
    print(rdq.weather_datapoints_without_five_forecasts(city, 'temp', end_dt=dt.datetime.combine(dt.date.today(), dt.time.min, dt.timezone.utc)).groupby('today_y').count()['_merge'])

london
today_y
2019-11-28    7
2019-11-29    8
2019-11-30    8
2019-12-01    8
2019-12-02    8
             ..
2023-03-13    8
2023-03-14    8
2023-03-15    8
2023-03-21    8
2023-03-22    8
Name: _merge, Length: 700, dtype: int64
madrid
today_y
2020-06-11    2
2020-06-12    8
2020-06-13    8
2020-06-14    8
2020-06-15    8
             ..
2023-03-13    8
2023-03-14    8
2023-03-15    8
2023-03-21    8
2023-03-22    8
Name: _merge, Length: 652, dtype: int64
saopaulo
today_y
2020-06-11    2
2020-06-12    8
2020-06-13    8
2020-06-14    8
2020-06-15    8
             ..
2023-03-13    8
2023-03-14    8
2023-03-15    8
2023-03-21    8
2023-03-22    8
Name: _merge, Length: 637, dtype: int64
sydney
today_y
2020-06-11    2
2020-06-12    8
2020-06-13    8
2020-06-14    8
2020-06-15    8
             ..
2023-03-13    8
2023-03-14    8
2023-03-15    8
2023-03-21    8
2023-03-22    8
Name: _merge, Length: 641, dtype: int64
newyork
today_y
2020-06-11    2
2020-06-12    8
2020-06-13    8
2020-06-14

### Missing forecasts
And here we can see that, as predicted, Madrid and Sao Paulo did not record those forecasts.

In [165]:
for city in rconf.cities.values():
    print(city.name)
    print(rdq.missing_forecast_datapoints(city, end_dt=dt.datetime.combine(dt.date.today(), dt.time.min, dt.timezone.utc)))

london
['2021-06-12' '2021-06-28' '2021-06-29' '2021-06-30' '2021-07-01'
 '2021-07-02' '2021-07-03' '2021-07-04' '2021-07-05' '2021-07-06'
 '2021-07-07' '2021-07-08' '2021-07-09' '2021-07-10' '2021-07-11'
 '2021-07-12' '2021-07-13' '2021-07-14' '2021-07-15' '2021-07-16'
 '2021-07-17' '2021-07-18' '2021-07-19' '2021-07-20' '2021-07-21'
 '2021-07-22' '2021-07-23' '2021-07-24' '2021-07-25' '2021-07-26'
 '2021-07-27' '2021-07-28' '2021-07-29' '2021-07-30' '2021-07-31'
 '2021-08-01' '2021-08-02' '2021-08-03' '2021-08-04' '2021-08-05'
 '2021-08-06' '2021-08-07' '2021-08-08' '2021-08-09' '2021-08-10'
 '2021-08-11' '2021-08-12' '2021-08-13' '2021-08-14' '2021-08-15'
 '2021-08-16' '2021-08-17' '2021-08-18' '2021-08-19' '2021-08-20'
 '2021-08-21' '2021-08-22' '2021-08-23' '2021-08-24' '2021-08-25'
 '2021-08-26' '2021-08-27' '2021-08-28' '2021-08-29' '2021-08-30'
 '2021-08-31' '2021-09-01' '2021-09-02' '2021-09-03' '2021-09-04'
 '2021-09-05' '2021-09-06' '2021-09-07' '2021-09-08' '2021-09-09'
 '2

<a id="actual_vs_forecast"></a>
## Actual vs Forecast temperature

In [170]:
fig, ax = plt.subplots()
@widgets.interact(days=(1, 60, 1), city_name=[city.name for city in rconf.cities.values()], tn=['t1','t2','t3','t4','t5'], weather_var=[wvar for wvar in rconf.weather_variables.values()])
def plot_actual_vs_forecast(days: int, city_name: str, tn: str, weather_var: str): 
    plt.grid(True)
    city = rconf.cities[city_name]    
    join_data_df = rdq.load_csv_files(city, weather_var)["join_data_df"]
    N = join_data_df.shape[0]
    max_x = N
    min_x = max_x-(rconf.day_factor*days)
    print(f"min_x={min_x}")
    print(f"max_x={max_x}")
    max_y = max(max(join_data_df[weather_var][min_x:max_x]), max(join_data_df[tn][min_x:max_x]))
    min_y = min(min(join_data_df[weather_var][min_x:max_x]), min(join_data_df[tn][min_x:max_x]))
    print(f"min_y={min_y}")
    print(f"max_y={max_y}")
    x = np.linspace(0,max_x-min_x,max_x-min_x)
    plt.xlim(0, max_x-min_x)
    plt.ylim(min_y, max_y)
    [l.remove() for l in ax.lines]
    [l.remove() for l in ax.lines]
    plt.plot(x, join_data_df[weather_var][min_x:max_x], label=f'actual {weather_var}', color='green', marker="o")
    plt.plot(x, join_data_df[tn][min_x:max_x], label=tn, color='red', marker='*')
    plt.title(f"{city_name}: t vs {tn} (last {days} days)")
    plt.legend()
#     ax.show()
#     print([l for l in ax.lines])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=30, description='days', max=60, min=1), Dropdown(description='city_name'…

<a id="metrics_comparison"></a>
## Metrics comparison

In all cases, `medae < mae < rmse`

In [167]:
fig, ax = plt.subplots()

@widgets.interact(city_name=[city.name for city in rconf.cities.values()], weather_var=[wvar for wvar in rconf.weather_variables.values()])
def plot_metrics(city_name: str, weather_var: str):
    plt.grid(True)
    city = rconf.cities[city_name]    
    metrics_df = rdq.load_csv_files(city, weather_var)["metrics_df"]
    # plt.rcParams['figure.figsize'] = (7,3)
    x = np.linspace(0,1,5)
    ax.set_xticks(x)
    ax.set_xticklabels(['t5', 't4', 't3', 't2', 't1'])
    max_y = max(max(metrics_df['mae']), max(metrics_df['rmse']), max(metrics_df['medae']))
    min_y = min(min(metrics_df['mae']), min(metrics_df['rmse']), min(metrics_df['medae']))
    print(f"min_y={min_y}")
    print(f"max_y={max_y}")
    plt.ylim(min_y, max_y)
    [l.remove() for l in ax.lines]
    [l.remove() for l in ax.lines]
    [l.remove() for l in ax.lines]
    plt.plot(x, metrics_df['mae'], label='mae', color='blue', marker='o')
    plt.plot(x, metrics_df['rmse'], label='rmse', color='grey', marker='^')
    plt.plot(x, metrics_df['medae'], label='medae', color='red', marker='*')
    plt.title(f"metrics - {city_name}")
    plt.legend()
#     plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='city_name', options=('london', 'madrid', 'saopaulo', 'sydney', 'ne…

In [168]:
fig, ax = plt.subplots()

@widgets.interact(city_name=[city.name for city in rconf.cities.values()], weather_var=[wvar for wvar in rconf.weather_variables.values()])
def plot_scaled_error(city_name: str, weather_var: str):
    plt.grid(True)
    city = rconf.cities[city_name]    
    metrics_df = rdq.load_csv_files(city, weather_var)["metrics_df"]
    x = np.linspace(0,1,5)
    ax.set_xticks(x)
    ax.set_xticklabels(['t5', 't4', 't3', 't2', 't1'])
    [l.remove() for l in ax.lines]
    [l.remove() for l in ax.lines]
    plt.plot(x, metrics_df['mase'], label='mase', color='blue', marker='o')
    plt.plot(x, metrics_df['mase1y'], label='mase1y', color='black', marker='*')
#     plt.plot(x, df["metrics_df"]['mase1y_avg'], label='mase1y_avg', color='black', marker='o')
    plt.plot(x, np.ones(5), label='1', color='red')
    plt.title(f"Mean Absolute Scaled Error - {city_name}")
    plt.legend()
#     plt.show()
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='city_name', options=('london', 'madrid', 'saopaulo', 'sydney', 'ne…

<a id="cities_comparison"></a>
## Cities comparison

In [169]:
fig, ax = plt.subplots()

@widgets.interact(metric=['mase', 'mae', 'rmse', 'medae'], weather_var=[wvar for wvar in rconf.weather_variables.values()])
def plot_cities(metric, weather_var):
    plt.grid(True)
    # plt.rcParams['figure.figsize'] = (7,3)
    colors = ['blue', 'red', 'green', 'black', 'purple']
    x = np.linspace(0,1,5)
    ax.set_xticks(x)
    ax.set_xticklabels(['t5', 't4', 't3', 't2', 't1'])
    idx = 0
#     plt.plot(x, np.ones(5), label='1', color='red')
    for city in rconf.cities.values():
        [l.remove() for l in ax.lines]
    for city in rconf.cities.values():
        files = rdq.load_csv_files(city, weather_var)    
        plt.plot(x, files["metrics_df"][metric], label=city.name, marker='o')
        idx += 1
    plt.title(metric)
    plt.legend()
#     plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='metric', options=('mase', 'mae', 'rmse', 'medae'), value='mase'), …

In [174]:
join_data_df = rdq.load_csv_files(rconf.cities['madrid'], 'temp')["join_data_df"]

In [177]:
join_data_df.iloc[2849]

temp          14.04
dt       1632117600
today    2021-09-20
t5            18.04
t4            16.88
t3            17.21
t2            16.98
t1            16.73
Name: 2849, dtype: object